In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad
from scipy.interpolate import interp1d

# Given data
column_names = ["time (s)", "Tracer concentration (ppm)"]

data_v_p6 = [  # Data for v = 0.6 dm^3/s
    (0, 0),
    (0.5, 8),
    (1, 640),
    (1.5, 951),
    (2, 495),
    (2.5, 215),
    (3, 103),
    (4, 34),
    (5, 11),
    (6, 7),
    (7, 0),
]
data_v_p6 = np.array(
    data_v_p6, dtype={"names": column_names, "formats": [float, float]}
)

data_v_p2 = [  # Data for v = 0.2 dm^3/s
    (0, 0),
    (0.5, 0),
    (1, 0),
    (1.5, 0),
    (2, 0),
    (2.5, 61),
    (3, 270),
    (3.5, 475),
    (4, 605),
    (4.5, 659),
    (5, 614),
    (6, 396),
    (7, 227),
    (8, 130),
    (9, 80),
    (10, 47),
    (12, 26),
    (14, 12),
    (16, 0),
]
data_v_p2 = np.array(
    data_v_p2, dtype={"names": column_names, "formats": [float, float]}
)

# working with data for v = 0.6 dm^3/s
# replace data_v_p6 with data_v_p2 to change the data set
# to v = 0.2 dm^3/s (also need to change Q)

# Flow rate
Q = 0.6  # dm^3/min

t = data_v_p6["time (s)"]
c = data_v_p6["Tracer concentration (ppm)"]

# Normalize concentration to calculate E(t)
integral_c = np.trapz(c, t)
et = c / integral_c

# Interpolation functions
et_interp = interp1d(t, et, kind="cubic", fill_value="extrapolate")

# Define cumulative distribution F(t)
def f_interp(t):
    return np.array([quad(et_interp, 0, ti, limit=1000)[0]
    for ti in np.atleast_1d(t)])

# Mean residence time function
tau_func = lambda t: t * et_interp(t)

# Variance function
variance_func = lambda t, tm: (t - tm) ** 2 * et_interp(t)

# Skewness function
skewness_func = lambda t, tm: (t - tm) ** 3 * et_interp(t)

# Calculate mean residence time (t_m)
tau, _ = quad(tau_func, 0, np.max(t))

# Calculate variance (σ²)
variance, _ = quad(variance_func, 0, np.max(t), args=(tau,))

# Calculate skewness (
sigma = variance**0.5
fac = 1.0 / (sigma**1.5)
integral, _ = quad(skewness_func, 0, np.max(t), args=(tau,))
skewness = fac * integral

# Calculate specific time fractions
# uncomment and adopt the following line as per required
# When you need to find integral till infinity,
# in place of infinity use np.max(t)
# fraction_2_to_4, _ = quad(et_interp, 2, 4)

internal_age = lambda t, tm: (1 / tm) * (1 - f_interp(t))
mean_internal_age, _ = quad(lambda t: internal_age(t, tau), 0, np.max(t))

# Reactor volume calculation

t_plot = np.linspace(0, np.max(t), 1000)
et_plot = et_interp(t_plot)

plt.scatter(t, et, label='E(t) experimental')
plt.plot(t_plot, et_plot, label='E(t) fitted')
plt.xlabel('Time (s)')
plt.ylabel('E(t)')
plt.xlim(np.min(t_plot), np.max(t_plot))
plt.legend()
plt.show()

it_plot = internal_age(t_plot, tau)

plt.plot(t_plot, it_plot, label='I(t)')
plt.xlabel('Time (min)')
plt.ylabel('I(t)')
plt.xlim(np.min(t_plot), np.max(t_plot))
plt.legend()
plt.show()

